# Index Stock Performance

In [ ]:
!pip install requests pandas yfinance investpy lxml beautifulsoup4 polygon-api-client selenium webdriver-manager

In [19]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor


## Fetch index components

In [24]:
def fetch_components_slickcharts(index):
    base_url = "https://slickcharts.com/"
    index_map = {
        'QQQ': 'nasdaq100',
        'SPX': 'sp500',
        'DOW': 'dowjones',
        'IWM': 'russell2000'
    }
    if index.upper() not in index_map:
        raise ValueError("Index must be either 'QQQ', 'SPX', 'DOW', or 'IWM'")
    
    url = f"{base_url}{index_map[index.upper()]}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    table = soup.find('table', {'class': 'table'})
    
    data = []
    headers = []

    for i, row in enumerate(table.find_all('tr')):
        cols = [ele.text.strip() for ele in row.find_all('td')]
        if i == 0:
            headers = [ele.text.strip() for ele in row.find_all('th')]
        else:
            if len(cols) > 1:  # Ensure there is data in the row
                data.append(cols)
    
    df = pd.DataFrame(data, columns=headers)
    return df


def print_all_company_details_compact(df):
    # Print DataFrame columns for debugging
    print("DataFrame columns:", df.columns)
    
    # Convert 'Weight' or 'Portfolio %' column to numeric if it exists
    weight_column = 'Weight' if 'Weight' in df.columns else 'Portfolio %' if 'Portfolio %' in df.columns else None
    if weight_column:
        df[weight_column] = df[weight_column].str.replace('%', '').astype(float)
        df = df.sort_values(by=weight_column, ascending=False)
    
    # Get the maximum length of each column for formatting
    col_widths = [max(df[col].astype(str).map(len).max(), len(col)) for col in df.columns]
    format_str = ' | '.join([f'{{:<{width}}}' for width in col_widths])
    
    print(format_str.format(*df.columns))
    print('-' * (sum(col_widths) + 3 * (len(col_widths) - 1)))
    
    for index, row in df.iterrows():
        print(format_str.format(*row))

# Example usage:
# Parameters
index = 'qqq'  # Change to 'SPX' for S&P 500 or 'DOW' for Dow Jones

# Fetch data from SlickCharts
df = fetch_components_slickcharts(index)

# Print details of all companies in a compact format
print_all_company_details_compact(df)

DataFrame columns: Index(['#', 'Company', 'Symbol', 'Portfolio%', 'Price', 'Chg', '% Chg', ''], dtype='object')
#   | Company                             | Symbol | Portfolio% | Price    | Chg    | % Chg    | 
-------------------------------------------------------------------------------------------------
1   | Microsoft Corp                      | MSFT   | 8.78%      | 460.36   | 1.08   | (0.23%)  | 
2   | Apple Inc                           | AAPL   | 8.59%      | 221.35   | 1.07   | (0.49%)  | 
3   | NVIDIA Corp                         | NVDA   | 7.90%      | 127.75   | 5.08   | (4.14%)  | 
4   | Amazon.com Inc                      | AMZN   | 5.31%      | 197.56   | -2.44  | (-1.22%) | 
5   | Broadcom Inc                        | AVGO   | 5.08%      | 1,723.10 | 65.62  | (3.96%)  | 
6   | Meta Platforms Inc                  | META   | 4.55%      | 509.99   | 0.49   | (0.10%)  | 
7   | Alphabet Inc                        | GOOGL  | 2.78%      | 185.84   | 0.60   | (0.32%)  | 
8   | 

## Calculate performance 

In [32]:
def calculate_performance(symbol, start_date, end_date):
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    
    if len(hist) == 0:
        return None
    
    start_price = hist['Close'].iloc[0]
    end_price = hist['Close'].iloc[-1]
    performance = (end_price - start_price) / start_price
    return performance

def get_performance_data(symbol, company, start_date, end_date):
    try:
        performance = calculate_performance(symbol, start_date, end_date)
        if performance is not None:
            return (symbol, company, performance)
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
    return None

def fetch_all_performance_data(df, start_date, end_date):
    performance_data = []
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = [executor.submit(get_performance_data, row['Symbol'], row['Company'], start_date, end_date) for index, row in df.iterrows()]
        for future in futures:
            result = future.result()
            if result:
                performance_data.append(result)
    
    performance_data.sort(key=lambda x: x[2], reverse=True)
    return performance_data

def display_top_bottom_stocks(df, period, top_n, bottom_n, start_year=None, end_year=None):
    end_date = datetime.now()
    
    if start_year and end_year:
        start_date = datetime(start_year, 1, 1)
        end_date = datetime(end_year, 12, 31)
    else:
        if period == 'ytd':
            start_date = datetime(end_date.year, 1, 1)
        else:
            delta = {
                '1d': 1,
                '5d': 5,
                '1m': 30,
                '6m': 182,
                '1y': 365,
                '2y': 365*2,
                '3y': 365*3,
                '5y': 365*5,
                '10y': 365*10,
                '20y': 365*20
            }[period]
            start_date = end_date - timedelta(days=delta)
            print("Start and end dates:", start_date, end_date)
    
    performance_data = fetch_all_performance_data(df, start_date, end_date)
    
    print(f"Top {top_n} Stocks:")
    for symbol, company, performance in performance_data[:top_n]:
        print(f"{symbol} ({company}): {performance*100:.1f}%")
    
    print(f"\nBottom {bottom_n} Stocks:")
    for symbol, company, performance in performance_data[-bottom_n:]:
        print(f"{symbol} ({company}): {performance*100:.1f}%")

In [33]:
# Example usage:
# Parameters
index = 'QQQ'  # Change to 'QQQ' for Nasdaq-100, 'DOW' for Dow Jones, 'IWM' for Russell 2000
period = '1m'  # Options: 'ytd', '1d', '5d', '1m', '6m', '1y', '2y', '3y', '5y', '10y', '20y'
top_n = 20
bottom_n = 20
start_year = None  # Example: 2004
end_year = None  # Example: 2024

# Fetch data from IEX Cloud
df = fetch_components_slickcharts(index)

# Display top and bottom performing stocks
display_top_bottom_stocks(df, period, top_n, bottom_n, start_year, end_year)

Start and end dates: 2024-06-03 11:42:27.338432 2024-07-03 11:42:27.338432
Top 20 Stocks:
TSLA (Tesla Inc): 31.2%
ADBE (Adobe Inc): 29.3%
ARM (ARM Holdings PLC ADR): 28.6%
AVGO (Broadcom Inc): 25.8%
CRWD (Crowdstrike Holdings Inc): 24.7%
DDOG (Datadog Inc): 20.5%
ADSK (Autodesk Inc): 17.6%
ZS (Zscaler Inc): 16.4%
INTU (Intuit Inc): 15.7%
PANW (Palo Alto Networks Inc): 15.6%
LRCX (Lam Research Corp): 14.9%
TEAM (Atlassian Corp): 14.3%
AAPL (Apple Inc): 13.5%
AMAT (Applied Materials Inc): 12.4%
AMZN (Amazon.com Inc): 12.1%
KLAC (KLA Corp): 11.3%
MSFT (Microsoft Corp): 11.1%
MDB (MongoDB Inc): 10.8%
CDNS (Cadence Design Systems Inc): 10.7%
GRAL (GRAIL Inc): 10.2%

Bottom 20 Stocks:
DASH (DoorDash Inc): -3.0%
IDXX (IDEXX Laboratories Inc): -3.1%
CMCSA (Comcast Corp): -3.6%
PEP (PepsiCo Inc): -3.7%
MCHP (Microchip Technology Inc): -3.7%
KDP (Keurig Dr Pepper Inc): -4.0%
PCAR (PACCAR Inc): -4.3%
CSGP (CoStar Group Inc): -4.9%
MNST (Monster Beverage Corp): -4.9%
TTWO (Take-Two Interactive Sof